In [1]:
from azure.identity import DefaultAzureCredential

token_credential = DefaultAzureCredential()
token = token_credential.get_token('https://management.azure.com/.default')
headers = {'Authorization': 'Bearer ' + token.token}

In [3]:
import os
import requests
import json
subscription_id = os.getenv("AZURE_SUBSCRIPTION_ID")
# Azure Management API endpoint
url = f"https://management.azure.com/subscriptions/{subscription_id}/providers/Microsoft.CognitiveServices/modelCapacities"

# Query parameters
params = {
    "api-version": "2024-10-01",
    "modelFormat": "OpenAI",
    "modelName": "o3-mini",
    "modelVersion": "2025-01-31"
}

# You'll need to authenticate with Azure
# This typically requires an access token from Azure AD
headers = {
    "Authorization": "Bearer " + token.token,
    "Content-Type": "application/json"
}

# Make the GET request
response = requests.get(url, headers=headers, params=params)

# Print the response
print(f"Status Code: {response.status_code}")
if response.status_code == 200:
    # print(json.dumps(response.json(), indent=4))
    print("SUCCESS")
else:
    print(f"Error: {response.text}")

Status Code: 200
SUCCESS


In [5]:
import pandas as pd

# Get the JSON data from the response
data = response.json()

# Extract the value list from the JSON response
values = data.get('value', [])

# Create a DataFrame from the values list
if values:
    # Flatten the nested JSON structure if needed
    df = pd.json_normalize(values)
    # Display the DataFrame as a table
    # display(df)
else:
    print("No values found in the response")

In [6]:
df.columns

Index(['id', 'type', 'name', 'location', 'properties.model.format',
       'properties.model.name', 'properties.model.version',
       'properties.skuName', 'properties.availableCapacity'],
      dtype='object')

In [7]:
# Display selected columns from the dataframe
display(df[['name', 'location', 'properties.model.name', 'properties.model.version',
           'properties.skuName', 'properties.availableCapacity']])

,name,location,properties.model.name,properties.model.version,properties.skuName,properties.availableCapacity
0,GlobalStandard,australiaeast,o3-mini,2025-01-31,GlobalStandard,500
1,ProvisionedManaged,australiaeast,o3-mini,2025-01-31,ProvisionedManaged,360
2,GlobalProvisionedManaged,australiaeast,o3-mini,2025-01-31,GlobalProvisionedManaged,0
3,GlobalStandard,brazilsouth,o3-mini,2025-01-31,GlobalStandard,500
4,GlobalProvisionedManaged,brazilsouth,o3-mini,2025-01-31,GlobalProvisionedManaged,0
...,...,...,...,...,...,...
85,GlobalProvisionedManaged,italynorth,o3-mini,2025-01-31,GlobalProvisionedManaged,0
86,GlobalStandard,spaincentral,o3-mini,2025-01-31,GlobalStandard,500
87,DataZoneStandard,spaincentral,o3-mini,2025-01-31,DataZoneStandard,200
88,GlobalProvisionedManaged,spaincentral,o3-mini,2025-01-31,GlobalProvisionedManaged,0


In [8]:
import ipywidgets as widgets
from IPython.display import display, HTML

# Get unique locations from the dataframe
locations = sorted(df['location'].unique())

# Create a dropdown widget
location_dropdown = widgets.Dropdown(
    options=locations,
    description='Location:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Create a function to filter the data
def filter_by_location(location):
    filtered_df = df[df['location'] == location]
    display(HTML(f"<h3>Model capacity in {location}</h3>"))
    display(HTML("<h4>Model Name: o3-mini</h4>"))
    display(filtered_df[['properties.model.name','name', 'properties.skuName', 'properties.availableCapacity']])

# Create an interactive output
output = widgets.Output()

# Define what happens when the dropdown value changes
def on_location_change(change):
    with output:
        output.clear_output()
        filter_by_location(change.new)

location_dropdown.observe(on_location_change, names='value')

# Display the widget and initial output
display(location_dropdown)
display(output)

# Show initial data for the first location
with output:
    filter_by_location(locations[0])

Dropdown(description='Location:', layout=Layout(width='300px'), options=('australiaeast', 'brazilsouth', 'cana…

Output()